In [44]:
# TODO : TEST https://www.tensorflow.org/api_docs/python/tf/io/encode_jpeg

In [95]:

from pydicom.dataset import FileMetaDataset

In [96]:
from pydicom import dcmread, dcmwrite, Dataset, DataElement, FileDataset
from pydicom.encaps import encapsulate
import matplotlib.pyplot as plt
%matplotlib inline
from pathlib import Path
import numpy as np
from simplejpeg import is_jpeg, decode_jpeg, decode_jpeg_header, encode_jpeg

In [35]:
folder = Path('/home/anvar/work/data/for_jpeg')
uid = '1.2.826.0.1.3680043.2.135.737538.61533215.7.1607388345.750.77'

dcm_rgb_lossy = dcmread('gdcm_lossy.dcm')
dcm_rgb = dcmread(folder / uid / 'covid' / '1.2.826.0.1.3680043.9.7574.1.55.772.37.616.3.1614346583.426.3.62.dcm')

### New tags

- DerivationCodeSequence
- DerivationDescription
- LossyImageCompressionRatio
- LossyImageCompression
- SourceImageSequence

### Changed tags

- (0008, 0008) Image Type                          CS: ['DERIVED ', 'PRIMARY', 'AXIAL']
- (0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL']

---

- (0008, 1140) Referenced Image Sequence           SQ: <Sequence, length 1>
- (0008, 1140) Referenced Image Sequence           SQ: <Sequence, length 1>

---

- (0008, 0018) SOP Instance UID                    UI: 1.2.826.0.1.3680043.2.1143.9872100504363308511163387960299132440
- (0008, 0018) SOP Instance UID                    UI: 1.2.826.0.1.3680043.9.7574.1.55.772.37.616.3.1614346583.426.3.62

---

- (7fe0, 0010) Pixel Data                          OB: Array of 86754 elements
- (7fe0, 0010) Pixel Data                          OW: Array of 786432 elements


### Change file_meta!

In [97]:
from pydicom import Dataset, FileDataset, DataElement 
from pydicom.dataset import FileMetaDataset
from pydicom.encaps import encapsulate
from simplejpeg import encode_jpeg
# TODO : test https://www.tensorflow.org/api_docs/python/tf/io/encode_jpeg

def array_to_jpeg(pixel_array:np.ndarray, quality:int = 90) -> str:
    """Encodes RGB, uint8 array using JPEG.
    
    Parameters
    ---
    
    pixel_array - nd.array,
        3-channel RGB array
    
    quality - int,
        image quality, directly affects compression ratio, default is 90 (~1/9 compression).
        
    Returns:
    ---
    
    jpeg_img - byte string
    
    Warning: DO NOT use this to compress grayscale single-channel uint16 images! 
    """
    # TODO move warning to exception
    
    assert len(pixel_array.shape) == 3
    assert pixel_array.shape[2] == 3
    
    return encode_jpeg(pixel_array, quality=quality)

def _add_tags_jpeg(instance: Dataset, compression_ratio:float) -> Dataset:
    """Adds dicom tags required by JPEG Baseline (Process 1)."""
    
#     DerivationCodeSequence
#     SourceImageSequence
    instance.DerivationDescription = 'lossy jpeg compression'
    instance.LossyImageCompressionRatio = str(compression_ratio)
    instance.LossyImageCompression = '01'
    instance[('0011', '0002')] = DataElement(('0011', '0002'), 'LO', '506f7765726564206279204149')
    return instance

# TODO: refactor/generalize .io.set_file_meta
def _update_header_jpeg(instance: Dataset) -> Dataset:
    """Updates dicom header required by JPEG Baseline (Process 1)."""
    meta = FileMetaDataset()
    meta.MediaStorageSOPClassUID = instance.SOPClassUID
    meta.MediaStorageSOPInstanceUID = instance.SOPInstanceUID
    meta.TransferSyntaxUID = '1.2.840.10008.1.2.4.50'# JPEG Baseline (Process 1)
    instance.is_implicit_VR = False
    instance.is_little_endian = True
    # TODO: Optional Implementation Class UID, Implementation Version Name, Source Application Entity Title
    instance.file_meta = meta
    instance.preamble = b'\x00' * 128
    return instance

def compress_frame(instance: Dataset, quality:int = 90, change_sop_uid=False) -> Dataset:
    # TODO: do we need to modify pixel-related tags since jpeg slightly messes up min-max etc.?
    """
    Compresses input instance, modifying PixelData and all required metadata tags.
    """
    
    compressed = encapsulate([array_to_jpeg(instance.pixel_array, quality)])
    ratio = len(instance.PixelData) / len(compressed)
    
    instance.PixelData = compressed
    instance.ImageType[0] = 'DERIVED'
    
        
    if change_sop_uid: # first change sop_uid if needed, next update header
        pass
    
    _add_tags_jpeg(instance, compression_ratio=ratio)
    _update_header_jpeg(instance)
    
    return instance

def compress_series(series):
    """
    Compresses input dicom Series.
    """
    for frame in series:
        yield compress_frame(frame)

In [98]:
dcm_rgb_lossy_manual = compress_frame(dcm_rgb)

In [99]:
dcmwrite('lossy_manual.dcm', dcm_rgb_lossy_manual)

In [101]:
from tqdm.notebook import tqdm

In [104]:
folder = Path('/home/anvar/work/data/for_jpeg')
uid = '1.2.826.0.1.3680043.2.135.737538.61533215.7.1607388345.750.77'

for frame in tqdm((folder / uid / 'covid').glob('*.dcm')):
    instance = dcmread(frame)
    try:
        compressed = compress_frame(instance)
        dcmwrite(folder / uid / 'manual_jpeg' / f'{instance.SOPInstanceUID}.dcm', compressed)
    except AttributeError:
        print(instance.SOPInstanceUID)
# dcm_rgb = dcmread(folder / uid / 'covid' / '1.2.826.0.1.3680043.9.7574.1.55.772.37.616.3.1614346583.426.3.62.dcm')

1.2.826.0.1.3680043.9.7574.1.55.772.37.616.3.1614346589.634.1.25

